In [2]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, ToPILImage
from PIL import Image

In [3]:
transform_tensor_to_pil = ToPILImage()
transform_pil_to_tensor = ToTensor()

# Evaluation will load different dataset or different subsets

## Any data set may be loaded

In [4]:
def load_data():

    # Download training data from open datasets.
    training_data = datasets.FashionMNIST(
        root="data",
        train=True,
        download=True,
        transform=ToTensor(),
    )

    # Download test data from open datasets.
    test_data = datasets.FashionMNIST(
        root="data",
        train=False,
        download=True,
        transform=ToTensor(),
    )
    
    return training_data, test_data

## sizes may be modified

In [5]:
class ModifiedDataset(Dataset):
  def __init__(self,given_dataset,shrink_percent=10):
    self.given_dataset = given_dataset
    self.shrink_percent = shrink_percent
    
  def __len__(self):
    return len(self.given_dataset)

  def __getitem__(self,idx):
    img, lab = self.given_dataset[idx]

    # print (type(img))
    # print (img.shape)

    img2 = transform_tensor_to_pil(img.squeeze())

    # print (img2.size)
    
    new_w = int(img2.size[0]*(1-self.shrink_percent/100.0))
    new_h = int(img2.size[1]*(1-self.shrink_percent/100.0))

    # print (new_w, new_h)

    img3 = img2.resize((new_w,new_h))

    # print (img3.size)

    x = transform_pil_to_tensor(img3)

    # print (x.shape)

    return x,lab

In [6]:
training_data, test_data = load_data()
mod_train_data = ModifiedDataset(training_data)
mod_test_data = ModifiedDataset(test_data)

print (training_data[0][0].shape)
print (mod_train_data[0][0].shape)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

torch.Size([1, 28, 28])
torch.Size([1, 25, 25])


In [7]:
def create_dataloaders(training_data, test_data, batch_size=64):

    # Create data loaders.
    train_dataloader = DataLoader(training_data, batch_size=batch_size)
    test_dataloader = DataLoader(test_data, batch_size=batch_size)

    for X, y in test_dataloader:
        print(f"Shape of X [N, C, H, W]: {X.shape}")
        print(f"Shape of y: {y.shape} {y.dtype}")
        break
        
    return train_dataloader, test_dataloader

# Sample code invocation

In [31]:
examplerollnum = 'cs19b003'

examplerepo = examplerollnum + 'iittp/islcourse:midsem'


train_data_loader1 = None
test_data_loader1 = None
train_data_loader1, test_data_loader1 = create_dataloaders(mod_train_data, mod_test_data)
print("test_data_loader1", test_data_loader1.dataset)

entrypoints = torch.hub.list(examplerepo,force_reload=True)

print ("entrypoints", entrypoints)


model = torch.hub.load(examplerepo,'get_model',train_data_loader=train_data_loader1,n_epochs=5, force_reload=True)
config1 = [(1,10,(3,3),1,'same'), (10,3,(5,5),1,'same'), (3,1,(7,7),1,'same')]
# model = torch.hub.load(examplerepo,'get_model_advanced',train_data_loader=train_data_loader1,n_epochs=10, lr=1e-4,config=config1, force_reload=True)

print (model)


a,p,r,f1 = torch.hub.load(examplerepo,'test_model',model1=model,test_data_loader=test_data_loader1,force_reload=True)

print (a,p,r,f1)


Shape of X [N, C, H, W]: torch.Size([64, 1, 25, 25])
Shape of y: torch.Size([64]) torch.int64
test_data_loader1 <__main__.ModifiedDataset object at 0x7f2ec2987050>


Downloading: "https://github.com/cs19b003iittp/islcourse/zipball/midsem" to /root/.cache/torch/hub/midsem.zip
Downloading: "https://github.com/cs19b003iittp/islcourse/zipball/midsem" to /root/.cache/torch/hub/midsem.zip


Using cpu device
entrypoints ['Cs19b003NN', 'DataLoader', 'ToTensor', 'get_model', 'get_model_advanced', 'kali', 'loss_fn', 'test_model']
Using cpu device
X and y shape torch.Size([64, 1, 25, 25]) torch.Size([64])
lens: 64 1 625
Returning model... (rollnumber: cs03) Cs19b003NN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=625, out_features=625, bias=True)
    (1): ReLU()
    (2): Linear(in_features=625, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
  (softmax): Softmax(dim=None)
)
Cs19b003NN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=625, out_features=625, bias=True)
    (1): ReLU()
    (2): Linear(in_features=625, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
  (softmax): Softmax(dim=None)
)


Downloading: "https://github.com/cs19b003iittp/islcourse/zipball/midsem" to /root/.cache/torch/hub/midsem.zip


Using cpu device
test_data_loader size: 10000  num batch 157
X and y shape torch.Size([64, 1, 25, 25]) torch.Size([64])
X and y shape torch.Size([64, 1, 25, 25]) torch.Size([64])
X and y shape torch.Size([64, 1, 25, 25]) torch.Size([64])
X and y shape torch.Size([64, 1, 25, 25]) torch.Size([64])
X and y shape torch.Size([64, 1, 25, 25]) torch.Size([64])
X and y shape torch.Size([64, 1, 25, 25]) torch.Size([64])
X and y shape torch.Size([64, 1, 25, 25]) torch.Size([64])
X and y shape torch.Size([64, 1, 25, 25]) torch.Size([64])
X and y shape torch.Size([64, 1, 25, 25]) torch.Size([64])
X and y shape torch.Size([64, 1, 25, 25]) torch.Size([64])
X and y shape torch.Size([64, 1, 25, 25]) torch.Size([64])
X and y shape torch.Size([64, 1, 25, 25]) torch.Size([64])
X and y shape torch.Size([64, 1, 25, 25]) torch.Size([64])
X and y shape torch.Size([64, 1, 25, 25]) torch.Size([64])
X and y shape torch.Size([64, 1, 25, 25]) torch.Size([64])
X and y shape torch.Size([64, 1, 25, 25]) torch.Size([